This notebook shows the use of `PowerLawRates`. This is a class meant to make the use of
rates that depend on the redshift through a power law easier. It follows the API set out
by `population_param_abstracts.BaseRateDistributions`.

In [1]:
from varpop import PowerLawRates
import varpop
print(varpop.__version__)

0.1.3dev11


In [2]:
import numpy as np

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

## Instantiate the class

In [4]:
rng = np.random.RandomState(1)
pl = PowerLawRates(rng, sky_area=5, zlower=1.0e-8, zhigher=1.2, num_bins=200, 
                   sky_fraction=None, zbin_edges=None, beta_rate=1.0)

The settings can be acccessed throughi the following attributes

In [5]:
pl.randomState

In [6]:
# This is the area of the sky used in sq degrees
pl.sky_area

5

## Bin Edges : 
This is the actual set of the bin edges used in the calculation.
These are the fundamental quantities irrespective of whether the edges are set
using `zbin_edges` or `zlower`, `zhigher`, `num_bins`

In [7]:
pl.zbin_edges

array([1.00000000e-08, 6.03016070e-03, 1.20603114e-02, 1.80904621e-02,
       2.41206128e-02, 3.01507635e-02, 3.61809142e-02, 4.22110649e-02,
       4.82412156e-02, 5.42713663e-02, 6.03015170e-02, 6.63316677e-02,
       7.23618184e-02, 7.83919691e-02, 8.44221198e-02, 9.04522706e-02,
       9.64824213e-02, 1.02512572e-01, 1.08542723e-01, 1.14572873e-01,
       1.20603024e-01, 1.26633175e-01, 1.32663325e-01, 1.38693476e-01,
       1.44723627e-01, 1.50753778e-01, 1.56783928e-01, 1.62814079e-01,
       1.68844230e-01, 1.74874380e-01, 1.80904531e-01, 1.86934682e-01,
       1.92964833e-01, 1.98994983e-01, 2.05025134e-01, 2.11055285e-01,
       2.17085435e-01, 2.23115586e-01, 2.29145737e-01, 2.35175887e-01,
       2.41206038e-01, 2.47236189e-01, 2.53266340e-01, 2.59296490e-01,
       2.65326641e-01, 2.71356792e-01, 2.77386942e-01, 2.83417093e-01,
       2.89447244e-01, 2.95477394e-01, 3.01507545e-01, 3.07537696e-01,
       3.13567847e-01, 3.19597997e-01, 3.25628148e-01, 3.31658299e-01,
      

The rate at each of these bins (at the mid point) of the redshifts defining the edges is given by the `volumetric_rate` and used for calculating the number of sources in the bin.

In [8]:
pl.volumetric_rate(np.array([0.1, 0.5, 1.0]))

array([2.59183583e-05, 3.53432159e-05, 4.71242879e-05])

### z_sample_size:
The expected number of TDA sources in each of the redshift bins is given by the following expression. This could be improved by first sampling `z_samples` and then binning which I intend to add in time. The problem with the current method is demonstrated below, but it is fine for small bins

In [9]:
pl.z_sample_sizes

array([2.26416006e-03, 1.57728584e-02, 4.25807623e-02, 8.24596568e-02,
       1.35181729e-01, 2.00519647e-01, 2.78246638e-01, 3.68136568e-01,
       4.69964010e-01, 5.83504325e-01, 7.08533727e-01, 8.44829357e-01,
       9.92169349e-01, 1.15033290e+00, 1.31910032e+00, 1.49825313e+00,
       1.68757407e+00, 1.88684721e+00, 2.09585798e+00, 2.31439321e+00,
       2.54224122e+00, 2.77919186e+00, 3.02503654e+00, 3.27956830e+00,
       3.54258185e+00, 3.81387359e+00, 4.09324170e+00, 4.38048613e+00,
       4.67540869e+00, 4.97781303e+00, 5.28750471e+00, 5.60429125e+00,
       5.92798210e+00, 6.25838873e+00, 6.59532463e+00, 6.93860532e+00,
       7.28804843e+00, 7.64347365e+00, 8.00470281e+00, 8.37155987e+00,
       8.74387093e+00, 9.12146428e+00, 9.50417038e+00, 9.89182189e+00,
       1.02842537e+01, 1.06813028e+01, 1.10828087e+01, 1.14886127e+01,
       1.18985588e+01, 1.23124929e+01, 1.27302633e+01, 1.31517205e+01,
       1.35767174e+01, 1.40051089e+01, 1.44367523e+01, 1.48715072e+01,
      

## z_samples
The actual samples of redshift can be obtained through 


In [10]:
pl.z_samples

array([0.06971175, 0.07641245, 0.08051717, ..., 1.19437457, 1.19677256,
       1.19961599])

    One can compare the size of `z_samples` and the expected size, `z_sample_sizes.sum()`

In [11]:
pl.z_sample_sizes.sum()

6910.968080056728

# A Comparison for SN

Let us choose the the canonical values for SNIa rates `alpha_rate` = 2.6e-5, and `beta_rate` = 1.0 (there are cases where this number is reported to be `1.5`, but we will ignore that for the simplicity of the limiting case. This means that the number density in each redshift bin is 

```n(z) = volumetric rate(z) * survey Time / (1 + z) = alpha_rate * survey Time * (h/0.7)^3``` 

Thus the expected number at z < 1.2  is Comoving Vol (z=1.2) * sky_fraction * alpha * (h/0.7)**3 and should match `pl.z_sample_size.sum()` for a sanity check

In [12]:
pl.cosmology.comoving_volume(1.2).value * ((5* np.radians(1)**2)/np.pi/4.) * pl.alpha_rate * 10. * (pl.cosmology.h/0.7)**3

6910.968080056729

In [13]:
pl.z_sample_sizes.sum()

6910.968080056728